# Clasificación
## Árboles de decisión 

Vamos a usar el dataset del [Titanic](https://www.kaggle.com/c/titanic/data), que contiene los datos de los pasajeros del Titanic, famoso barco que se hundió en su primer viaje en 1912. Dicho dataset contiene los datos de los pasajeros así como información sobre si sobrevivieron o no.

**Descripción de los datos**

* superviviente 	 El pasajero sobrevivió 	0 = No, 1 = Si

* clase_billete 	 Clase de camarote 	1 = Primera clase, 2 = Segunda, 3 = Tercera clase (la más pobre)

* genero 	         hombre/mujer 	

* edad 	

* n_hermanos_esposos Número de hermanos o pareja a bordo del Titanic

* n_hijos_padres     Número de hijos o padres a bordo del Titanic 

* precio_billete 	

* puerto_salida      Puerto donde el pasajero tomó el barco (C=Cherbourg, Q=Queenstown, S=Southampton)


In [1]:
import pandas as pd

In [2]:
datos = pd.read_csv("data/titanic.csv")

In [3]:
datos.head()

,superviviente,clase_billete,genero,edad,n_hermanos_esposos,n_hijos_padres,precio_billete,puerto_salida
0,0,3,hombre,22.0,1,0,7.2500,S
1,1,1,mujer,38.0,1,0,71.2833,C
2,1,3,mujer,26.0,0,0,7.9250,S
3,1,1,mujer,35.0,1,0,53.1000,S
4,0,3,hombre,35.0,0,0,8.0500,S


Los algoritmos de creación de árboles están en el submódulo de `sklearn.tree` 

En cuanto al tipo de algoritmo para crear árboles, scikit-learn [usa una versión optimizada del algoritmo CART](http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart) *(Classification and Regression Trees)*, que  permite usar árboles de decisión tanto para problemas de clasificación como de regresión.

In [4]:
from sklearn import tree
from sklearn.model_selection import cross_val_score

In [5]:
from sklearn import preprocessing

In [6]:
columnas_categoricas = ["genero", "puerto_salida"]

In [7]:
codificador_binario = preprocessing.LabelBinarizer()

In [8]:
codificador_binario

LabelBinarizer()

In [9]:
datos_categoricos = pd.get_dummies(datos[columnas_categoricas], drop_first=True)

In [10]:
datos_categoricos.head()

,genero_mujer,puerto_salida_Q,puerto_salida_S
0,0,0,1
1,1,0,0
2,1,0,1
3,1,0,1
4,0,0,1


In [11]:
pasajeros = (
    pd.concat([
        datos.drop(columnas_categoricas, axis=1),
        datos_categoricos
    ],axis=1
    )
)
pasajeros.edad = pasajeros.edad.fillna(pasajeros.edad.mean())

In [12]:
pasajeros.head()

,superviviente,clase_billete,edad,n_hermanos_esposos,n_hijos_padres,precio_billete,genero_mujer,puerto_salida_Q,puerto_salida_S
0,0,3,22.0,1,0,7.2500,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,1
3,1,1,35.0,1,0,53.1000,1,0,1
4,0,3,35.0,0,0,8.0500,0,0,1


In [13]:
arbol = tree.DecisionTreeClassifier()

X = pasajeros.drop("superviviente", axis=1)
y = pasajeros.superviviente

In [14]:
arbol.fit(X, y)

DecisionTreeClassifier()

In [15]:
cross_val_score(arbol, X, y, scoring="roc_auc", cv=10)
# cross_val_score(arbol, X, y, scoring="roc_auc", cv=10).mean()

array([0.71298701, 0.7540107 , 0.66417112, 0.75989305, 0.87727273,
       0.74064171, 0.7631016 , 0.73262032, 0.85213904, 0.79814815])

Una funcionalidad interesante que tienen los arboles de decision en `sklearn` es que se pueden visualizar con `Graphviz`. Para ello tenemos que instalar la libreria con `pip install graphviz`.

In [16]:
import matplotlib
%matplotlib inline

In [ ]:
# import os
# os.environ["PATH"] += os.pathsep + 'C:/Users/Alfy/Downloads/graphviz-2.38/release/bin'

In [17]:
import os
os

<module 'os' from 'C:\\Users\\Jhon\\anaconda3\\lib\\os.py'>

In [18]:
conda remove graphviz
conda install python-graphviz

SyntaxError: invalid syntax (<ipython-input-18-d056e975b02c>, line 1)

In [47]:
import graphviz

def dibujar_arbol(arbol):
    dot_data = tree.export_graphviz(arbol, out_file=None, 
                         feature_names=pasajeros.drop("superviviente", axis=1).columns,  
                         filled=True, 
                         impurity=False,
                         rounded=True,  
                         special_characters=True)  
    
    graph = graphviz.Source(dot_data)
    graph.format = 'png'
    graph.render('arbol',view=True)

In [49]:
import os


In [52]:
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'


In [54]:
C:\Program Files (x86)\Graphviz2.38\bin

SyntaxError: unexpected character after line continuation character (<ipython-input-54-a9c0b78bf033>, line 1)

In [56]:
dibujar_arbol(arbol)

ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tpng', '-O', 'arbol'], make sure the Graphviz executables are on your systems' PATH

Alternativamente, se puede exportar el árbol y abrirlo posteriormente con graphviz desde la terminal (o desde la página http://webgraphviz.com/ que renderiza archivos de graphviz)

In [31]:
tree.export_graphviz(arbol, out_file="arbol.dot")

Otra funcionalidad que tienen los árboles de decisión en sklearn es que nos dan una indicación de la importancia de cada variable en el modelo, almacenada en el atributo `feature_importances_`. Calcula la importancia en función de la ganancia de información de cada variable, es decir, que variables separan mejor las distintas clases.

In [30]:
arbol.feature_importances_

array([0.12020515, 0.23690403, 0.04111129, 0.03773797, 0.23518282,
       0.30933519, 0.00745827, 0.01206527])

In [32]:
dict(zip(
    pasajeros.drop("superviviente", axis=1),
    arbol.feature_importances_
))

{'clase_billete': 0.12020515499904108,
 'edad': 0.23690403230078233,
 'n_hermanos_esposos': 0.04111129323328178,
 'n_hijos_padres': 0.03773797160389421,
 'precio_billete': 0.23518281622501772,
 'genero_mujer': 0.30933518862833864,
 'puerto_salida_Q': 0.0074582732214090106,
 'puerto_salida_S': 0.012065269788235218}

Por ejemplo, en este árbol vemos que las variables que tienen más peso a la hora de decidir si un pasajero sobrevive o no son la edad, el género y el precio del billete (¡las mujeres y los niños primero!).

In [33]:
tree.DecisionTreeClassifier?

Éstos son los parámetros más importantes para los modelos DecisionTreeClassifier de sklearn:

* criterion : El criterio para calcular la reducción de impureza (ganancia de información) al hacer una partición. Se puede elegir entre `gini`, o `entropy` 

* max_depth : La profundidad máxima del árbol. Definimos profundidad como el número de nodos que atraviesa una observación *(cuantas "preguntas" se le hacen)*.

* max_features:  El máximo numero de particiones potenciales se consideran al evaluar un nodo.

* max_leaf_nodes : Límite de hojas para el árbol.

* min_impurity_decrease : la ganancia de información mínima en un nodo para hacer una partición. (Si no hay ninguna partición que cumpla este criterio, se para el desarrollo del árbol en dicho nodo).

* class_weight : Para clases imbalanceadas, podemos pasar el argumento class_weight, como un diccionario de `{clase: peso}` para que sklearn tenga en cuenta los pesos. Alternativamente, podemos pasar el string `balanced` para que sklearn genere pesos en función del número de muestras de cada clase.

Los árboles de decisión tienden a sobreajustar cuando tienen demasiada complejidad. Los siguientes parámetros ayudan a controlarla:

* min_samples_leaf : Mínimo número de observaciones en un nodo para considerar un nodo como una hoja. Por defecto es 1, ésto significa que sklearn solo para de hacer particiones cuando un nodo tiene sólo 1 observación.

* min_samples_split : Mínimo número de observaciones en un nodo para generar una partición. Por defecto es 2, esto significa que sklearn por defecto siempre va a intentar particionar un nodo con 2+ elementos.

In [34]:
arbol_simple = tree.DecisionTreeClassifier(max_depth=4)

In [35]:
arbol_simple.fit(X, y)

DecisionTreeClassifier(max_depth=4)

In [36]:
dibujar_arbol(arbol_simple)

ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tpng', '-O', 'arbol'], make sure the Graphviz executables are on your systems' PATH

In [37]:
cross_val_score(arbol_simple, X, 
                y, scoring="roc_auc", cv=10).mean()

0.8524582802818097

In [38]:
pasajeros.groupby("superviviente").size()

superviviente
0    549
1    342
dtype: int64

In [39]:
arbol_balanceado = tree.DecisionTreeClassifier(max_depth=3, class_weight="balanced")

In [40]:
arbol_balanceado.fit(X, y)

DecisionTreeClassifier(class_weight='balanced', max_depth=3)

In [41]:
dibujar_arbol(arbol_balanceado)

ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tpng', '-O', 'arbol'], make sure the Graphviz executables are on your systems' PATH

In [42]:
cross_val_score(arbol_balanceado, X, 
                y, scoring="roc_auc", cv=10).mean()

0.8615064652123475

In [43]:
cross_val_score(arbol_balanceado, X, y,
                scoring="precision", cv=10).mean()

0.7411115910638909

Además del algoritmo CART para generar árboles, scikit-learn también proporciona una clase de arboles llamada ExtraTreeClassifier, o Extremely Random Trees (Árboles Extremadamente Aleatorios). En estos árboles, en lugar de seleccionar en cada nodo la párticion que proporciona la mayor ganancia de información, ¡se decide una partición al azar!.

In [44]:
arbol_aleatorio = tree.ExtraTreeClassifier(max_features=1)

In [45]:
arbol_aleatorio.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

dibujar_arbol(arbol_aleatorio)

ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tpng', '-O', 'arbol'], make sure the Graphviz executables are on your systems' PATH

Los árboles aleatorios funcionan peor que los árboles de decisión por sí mismos, pero usados en grupo pueden funcionar mejor.

In [46]:
cross_val_score(arbol_aleatorio,X, y, scoring="roc_auc",
                cv=10).mean()

0.7616311292193645

Los árboles aleatorios funcionan peor que los árboles de decisión por sí mismos, pero usados en grupo pueden funcionar mejor.